In [1]:
# Install the package
%pip install -U crawl4ai

Note: you may need to restart the kernel to use updated packages.


> **Note:** The following commands require sudo/admin privileges to run. As Jupyter notebooks cannot input passwords, you may need to run these commands in a terminal outside of the notebook environment.


In [ ]:
# Run post-installation setup
!crawl4ai-setup

# Verify your installation
!crawl4ai-doctor

After running the above commands, you should see output similar to the following:

1. Post-installation setup:
   ```
   [COMPLETE] ● Database initialization completed successfully.
   [COMPLETE] ● Post-installation setup completed!
   ```

2. Installation verification:
   ```
   [COMPLETE] ● https://crawl4ai.com... | Status: True | Total: 17.10s
   [COMPLETE] ● ✅ Crawling test passed!
   ```

These messages indicate that the setup was successful and the installation is working correctly.


In [2]:
# Install pandas for easy view results in notebook
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Load .env file
from dotenv import load_dotenv
load_dotenv()

# logging
import logging
logging.getLogger("httpx").setLevel(logging.WARNING)

## Example 1: Using crawl4ai without LLM

The following code demonstrates how to use crawl4ai to crawl a website and extract links without using any language model:

```python
import asyncio
from crawl4ai import AsyncWebCrawler
import pandas as pd

async def crawl_website():
    async with AsyncWebCrawler(verbose=True, headless=True) as crawler:
        result = await crawler.arun(
            url="https://www1.hkej.com/features/topic/tag/%E8%99%9B%E5%B9%A3%E5%8B%95%E6%85%8B",
            bypass_cache=False,
            verbose=False,
        )

        # Get all internal links from the website
        df = pd.DataFrame(result.links["internal"])
        display(df)

asyncio.run(crawl_website())
```

This example crawls the specified URL and displays all internal links found on the page using a pandas DataFrame.


In [4]:
import asyncio
from crawl4ai import AsyncWebCrawler
import os
import json

import pandas as pd

In [5]:
# Try to Grapping news from hkej website

async with AsyncWebCrawler(verbose=True,headless=True) as crawler:
    result = await crawler.arun(
        url="https://www1.hkej.com/features/topic/tag/%E8%99%9B%E5%B9%A3%E5%8B%95%E6%85%8B",
        bypass_cache=False,
        verbose=False,
    )

    # Example 1: Get all links from the website
    df = pd.DataFrame(result.links["internal"])
    display(df)
    
    # Example 2: Get all news links from the website
    df_filtered = df[df['href'].str.contains('article', case=False, na=False)]
    display(df_filtered)

[INIT].... → Crawl4AI 0.4.247
[FETCH]... ↓ https://www1.hkej.com/features/topic/tag/%E8%99%9B... | Status: True | Time: 0.08s
[COMPLETE] ● https://www1.hkej.com/features/topic/tag/%E8%99%9B... | Status: True | Total: 0.10s


,href,text,title,base_domain
0,https://subscribe.hkej.com,訂閱 / 續訂,,hkej.com
1,https://subscribe.hkej.com/register,註冊,,hkej.com
2,https://subscribe.hkej.com/member/login?forwar...,登入,,hkej.com
3,https://www2.hkej.com/landing/index,,,hkej.com
4,https://www2.hkej.com/weather/weather.php?loca...,,,hkej.com
...,...,...,...,...
120,https://www2.hkej.com/info/privacy,私隱條款,,hkej.com
121,https://www2.hkej.com/info/disclaimer,免責聲明,,hkej.com
122,https://www.hkej.com/ratecard/html/index.html,廣告查詢,,hkej.com
123,https://www2.hkej.com/info/jobs,加入信報,,hkej.com


,href,text,title,base_domain
30,https://www1.hkej.com/features/article?q=%23%E...,,,hkej.com
31,https://www1.hkej.com/features/article?q=%23%E...,,,hkej.com
32,https://www1.hkej.com/features/article?q=%23%E...,,,hkej.com
33,https://www1.hkej.com/features/article?q=%23%E...,,,hkej.com
34,https://www1.hkej.com/features/article?q=%23%E...,,,hkej.com
35,https://www1.hkej.com/features/article?q=%23%E...,,,hkej.com
36,https://www1.hkej.com/features/article?q=%23%E...,,,hkej.com
37,https://www1.hkej.com/features/article?q=%23%E...,,,hkej.com
38,https://www1.hkej.com/features/article?q=%23%E...,,,hkej.com
39,https://www1.hkej.com/features/article?q=%23%E...,,,hkej.com


## Example 2: Using crawl4ai with LLM and Pydantic Schema

This example demonstrates how to use crawl4ai with a language model (LLM) and a Pydantic schema to extract specific information from a website.

```python
from crawl4ai import AsyncWebCrawler, LLMExtractionStrategy, CrawlerRunConfig, CacheMode

async def crawl_with_llm():
    async with AsyncWebCrawler(verbose=True, headless=True) as crawler:
        result = await crawler.arun(
            url="https://www1.hkej.com/features/topic/tag/%E8%99%9B%E5%B9%A3%E5%8B%95%E6%85%8B",
            config=crawler_config,
            verbose=False
        )
        
        # Display the extracted data
        for item in result.extracted_data:
            print(json.dumps(item, indent=2, ensure_ascii=False))

asyncio.run(crawl_with_llm())
```

This code uses the `LLMExtractionStrategy` with a custom Pydantic schema to extract specific information (title, image URL, article URL, and date) from news articles. The LLM is guided by a simple prompt to focus on these details during the extraction process.


In [6]:
from pydantic import BaseModel, Field
from crawl4ai import LLMExtractionStrategy, CrawlerRunConfig, CacheMode

class OpenAIModelFee(BaseModel):
    title: str = Field(..., description="News title.")
    image: str = Field(..., description="Url of the news thumbnail.")
    url: str = Field(..., description="Url of the news.")
    date: str = Field(..., description="Date of the news.")

deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
provider = f"azure/{deployment_name}"
api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
api_token = os.getenv("AZURE_OPENAI_API_KEY")
extra_args = {"temperature": 0.7}

llm_strategy = LLMExtractionStrategy(
            api_base = api_base,
            provider=provider,
            api_token=api_token,
            schema=OpenAIModelFee.model_json_schema(),
            extraction_type="schema",
            instruction="""From the crawled content, extract all the news title, image url, url and date.""",
            extra_args=extra_args,
            verbose=False
        )

crawler_config = CrawlerRunConfig(
        cache_mode=CacheMode.BYPASS,
        word_count_threshold=1,
        page_timeout=80000,
        extraction_strategy=llm_strategy,
        verbose=False
    )
    
async with AsyncWebCrawler() as crawler:
    result = await crawler.arun(
        url="https://www1.hkej.com/features/topic/tag/%E8%99%9B%E5%B9%A3%E5%8B%95%E6%85%8B",
        bypass_cache=False,
        verbose=False,
        config=crawler_config
    )
    # The JSON output is stored in 'extracted_content'
    data = json.loads(result.extracted_content)
    df = pd.DataFrame(data)
    display(df)

    llm_strategy.show_usage() 


[INIT].... → Crawl4AI 0.4.247
[FETCH]... ↓ https://www1.hkej.com/features/topic/tag/%E8%99%9B... | Status: True | Time: 3.91s
[SCRAPE].. ◆ Processed https://www1.hkej.com/features/topic/tag/%E8%99%9B... | Time: 39ms


/home/kan/workspaces/poc/genai-samples/.venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)
18:00:51 - LiteLLM:INFO: utils.py:2820 - 
LiteLLM completion() model= gpt-4o-mini; provider = azure
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o-mini; provider = azure
18:01:18 - LiteLLM:INFO: utils.py:952 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler


[EXTRACT]. ■ Completed for https://www1.hkej.com/features/topic/tag/%E8%99%9B... | Time: 31.54344184906222s
[COMPLETE] ● https://www1.hkej.com/features/topic/tag/%E8%99%9B... | Status: True | Total: 35.51s


,title,image,url,date,error
0,$TRUMP幣熱炒 存利益衝突,https://static.hkej.com/hkej/images/2025/01/20...,https://www1.hkej.com/features/article?q=%23%E...,2025年1月20日,False
1,HashKey Exchange去年交易量升85% 料今年達收支平衡,https://static.hkej.com/hkej/images/2025/01/15...,https://www1.hkej.com/features/article?q=%23%E...,2025年1月15日,False
2,Z世代寧買比特幣更勝置業,https://static.hkej.com/hkej/images/2025/01/15...,https://www1.hkej.com/features/article?q=%23%E...,2025年1月15日,False
3,歐股靠穩 比特幣失守9.2萬美元,https://static.hkej.com/hkej/images/2025/01/10...,https://www1.hkej.com/features/article?q=%23%E...,2025年1月10日,False
4,【ETF特搜】比特幣重上10萬美元 相關ETF升逾2%,https://static.hkej.com/hkej/images/2025/01/07...,https://www1.hkej.com/features/article?q=%23%E...,2025年1月7日,False
5,馬斯克X賬號改名 迷因幣爆炒,https://static.hkej.com/hkej/images/2025/01/03...,https://www1.hkej.com/features/article?q=%23%E...,2025年1月3日,False
6,比特幣獲唱好今年衝上20萬美元,https://static.hkej.com/hkej/images/2025/01/02...,https://www1.hkej.com/features/article?q=%23%E...,2025年1月2日,False
7,聯儲局官員戴利:加密貨幣並非黃金,https://static.hkej.com/hkej/images/2024/12/31...,https://www1.hkej.com/features/article?q=%23%E...,2024年12月31日,False
8,星洲年批13加密幣牌照 遠超香港,https://static.hkej.com/hkej/images/2024/12/27...,https://www1.hkej.com/features/article?q=%23%E...,2024年12月27日,False
9,比特幣瘋炒 2024最癲交易,https://static.hkej.com/hkej/images/2024/12/24...,https://www1.hkej.com/features/article?q=%23%E...,2024年12月24日,False



=== Token Usage Summary ===
Type                   Count
------------------------------
Completion             2,868
Prompt                12,430
Total                 15,298

=== Usage History ===
Request #    Completion       Prompt        Total
------------------------------------------------
1                 2,868       12,430       15,298
